In [1]:
import mlrun

# Create the 'LLM chatbot' AIPC project

In [3]:
project = mlrun.get_or_create_project(
    name="llm-text2seq-aipc",
    context="./",
    parameters={
        "source": "https://github.com/smartcommunitylab/ai_product_card_templates/tree/main/aipc_examples/text2seq",
    },
)
secrets = {"MINIO_URL": "", "MINIO_AK": "", "MINIO_SK": "", 
          "WANDB_ENTITY": "", "WANDB_PROJECT": "", 
           "WANDB_API_KEY": "", 
           "HF_TOKEN": ""}
project.set_secrets(secrets=secrets)

Project Source: https://github.com/smartcommunitylab/ai_product_card_templates/tree/main/aipc_examples/text2seq
> 2024-07-05 07:17:16,712 [info] Project loaded successfully: {'project_name': 'llm-text2seq-aipc'}


# Training function

In [11]:
# Training function
fn = project.set_function(
    image="mlrun/mlrun-gpu",
    name="training",
    func='functions/training.py',
    handler='training',
    kind="job",
    requirements = ["peft", "transformers", "datasets", "bitsandbytes", "trl", "accelerate", "wandb"]
)
project.build_function(fn)

> 2024-07-05 11:34:00,672 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2024-07-05 11:34:00,784 [info] Started building image: .mlrun/func-llm-text2seq-aipc-training:latest


The `overwrite_build_params` parameter default will change from 'False' to 'True' in 1.8.0.
The clone_target_dir attribute is deprecated in 1.6.2 and will be removed in 1.8.0. Use spec.build.source_code_target_dir instead.


INFO[0000] Retrieving image manifest mlrun/mlrun-gpu:1.6.2-rc1 
INFO[0000] Retrieving image mlrun/mlrun-gpu:1.6.2-rc1 from registry index.docker.io 
INFO[0002] Built cross stage deps: map[]                
INFO[0002] Retrieving image manifest mlrun/mlrun-gpu:1.6.2-rc1 
INFO[0002] Returning cached image manifest              
INFO[0002] Executing 0 build triggers                   
INFO[0002] Unpacking rootfs as cmd RUN echo 'Installing /empty/requirements.txt...'; cat /empty/requirements.txt requires it. 
INFO[0166] ARG HF_TOKEN=$HF_TOKEN                       
INFO[0166] ARG MINIO_AK=$MINIO_AK                       
INFO[0166] ARG MINIO_SK=$MINIO_SK                       
INFO[0166] ARG MINIO_URL=$MINIO_URL                     
INFO[0166] ARG WANDB_API_KEY=$WANDB_API_KEY             
INFO[0166] ARG WANDB_ENTITY=$WANDB_ENTITY               
INFO[0166] ARG WANDB_PROJECT=$WANDB_PROJECT             
INFO[0166] RUN echo 'Installing /empty/requirements.txt...'; cat /empty/requirements.txt 


BuildStatus(ready=True, outputs={'image': '.mlrun/func-llm-text2seq-aipc-training:latest'})

In [12]:
from kubernetes import client
tol = [
    client.V1Toleration(
        key='virtual-kubelet.io/provider',
        operator='Equal',
        value="k8sgpu",
        effect='NoExecute',
    ),
    client.V1Toleration(
        key='node.kubernetes.io/network-unavailable',
        operator='Exists',
        effect='NoSchedule',
    )
]
fn.with_node_selection(tolerations=tol,node_selector={"kubernetes.io/hostname": "k8s.gpu"})
fn.with_limits(mem="100G", cpu=10, gpus=1)

In [ ]:
fn.run()

# Serving function

In [ ]:
#Serving function
serving_fn = mlrun.code_to_function(
    "serving-llama2", 
    filename="serve2.py", 
    kind="serving", 
    image="mlrun/mlrun-gpu",
)
serving_fn.spec.build.commands = [
    "pip install torch peft transformers bitsandbytes accelerate minio",
]

In [ ]:
# define resources
serving_fn.spec.replicas = 1
from kubernetes import client
tol = [
    client.V1Toleration(
        key='nvidia.com/gpu',
        operator='Equal',
        value='a100',
        effect='NoSchedule',
    )
]
serving_fn.with_node_selection(tolerations=tol)
serving_fn.with_limits(gpus=1,mem="200G")

In [ ]:
serving_fn.add_model(
    "llama2",
    model_path=" ",
    model_name="meta-llama/Llama-2-7b-hf",
    adapter_path='checkpoint-400',
    class_name="ChatBot"
)
myproject.deploy_function(serving_fn)
myproject.save()

## Chat with the finetuned model

In [ ]:
text = "'You are a powerful text-to-SQL model. Your job is to answer questions about a database. \
You are given a question and context regarding one or more tables. \
    \n\nYou must output the SQL query that answers the question.\
    \n\n### Input:\nWho won the points classification when the teams classification winner was Lampre-Farnese? \
\n\n### Context:\nCREATE TABLE table_28092844_16 (points_classification_klasyfikacja_punktowa VARCHAR, teams_classification VARCHAR)\
\n\n### Response:\n"
skip_special_tokens=False
max_new_tokens=250
do_sample=False
sample = {
    "row": text, 
    "skip_special_tokens": skip_special_tokens, 
    "max_new_tokens": max_new_tokens,
    "do_sample": do_sample
}
response = serving_fn.invoke(path=f"/v2/models/llama2/infer", body={"inputs": [sample]})